In [3]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray


In [ ]:
def run_excavation_pipeline(
    monitordf,
    window=3,
    anomaly_quantile=0.90,
    persistence_days_rise=30,
    persistence_days_fall=60,
    min_persist_rise=2,
    min_persist_fall=2,
    max_gap_days_for_delta=30,
):
    """
    Improved temporal excavation detection pipeline.


    States:
      0 = non-excavated
      1 = transient anomaly / under observation
      2 = excavated (persistent)
    """
    df = monitordf.copy()
    df["time"] = pd.to_datetime(df["time"])
    df = df.sort_values(["y", "x", "time"])


    # ==== 1. Basic per-pixel NDVI time derivatives ====
    # Keep NDVI NaNs; do NOT drop rows here, they carry cloud info.
    df["NDVI_prev"] = df.groupby(["y", "x"], group_keys=False)["NDVI"].shift(1)
    df["time_prev"] = df.groupby(["y", "x"], group_keys=False)["time"].shift(1)
    df["delta_days"] = (df["time"] - df["time_prev"]).dt.days


    # Only compute deltaNDVI when both NDVI values exist and gap is not too large
    valid_delta_mask = (
        df["NDVI_prev"].notna()
        & df["NDVI"].notna()
        & df["delta_days"].notna()
        & (df["delta_days"] <= max_gap_days_for_delta)
    )
    df["deltaNDVI"] = np.nan
    df.loc[valid_delta_mask, "deltaNDVI"] = (
        df.loc[valid_delta_mask, "NDVI"] - df.loc[valid_delta_mask, "NDVI_prev"]
    )


    # Rolling slope (over valid NDVI values only)
    def rolling_slope(series):
        series = series.dropna()
        x = np.arange(len(series))
        if len(series) < 2:
            return np.nan
        return np.polyfit(x, series.values, 1)[0]


    df["NDVI_slope"] = (
        df.groupby(["y", "x"], group_keys=False)["NDVI"]
        .rolling(window=window, min_periods=2)
        .apply(rolling_slope, raw=False)
        .reset_index(level=[0, 1], drop=True)
    )


    # ==== 2. Anomaly thresholds ====
    # Anomaly threshold from monitoring distribution
    ANOMALY_THRESH = df["anomalyscore"].quantile(anomaly_quantile)


    # Per-pixel NDVI drop threshold, with global fallback if series too short
    # Compute per-pixel 0.10 quantile when there are enough valid deltas.
    per_pixel_q10 = (
        df.groupby(["y", "x"], group_keys=False)["deltaNDVI"]
        .quantile(0.10, interpolation="linear")
        .rename("delta_q10")
    )
    df = df.join(per_pixel_q10, on=["y", "x"])


    global_q10 = df["deltaNDVI"].quantile(0.10)  # typically negative
    if pd.isna(global_q10):
        global_q10 = -0.05  # safe fallback


    # Use per-pixel q10 where available, else global
    df["NDVI_DROP_THRESH"] = df["delta_q10"].fillna(global_q10)


    # Median temporal delta used to derive persistence lengths in time-steps
    valid_time_deltas = df["delta_days"].dropna()
    median_delta = valid_time_deltas.median() if not valid_time_deltas.empty else 10


    K_RISE = max(min_persist_rise, int(round(persistence_days_rise / median_delta)))
    K_FALL = max(min_persist_fall, int(round(persistence_days_fall / median_delta)))


    # ==== 3. Initial state based on anomaly + NDVI drop ====
    df["stateraw"] = 0
    transition_mask = (
        (df["anomalyscore"] >= ANOMALY_THRESH)
        & (df["deltaNDVI"] <= df["NDVI_DROP_THRESH"])
    )
    df.loc[transition_mask, "stateraw"] = 1


    # Explicit no-data flag: if NDVI is NaN, mark separately so it doesn’t affect persistence
    df["is_nodata"] = df["NDVI"].isna()


    # ==== 4. Persistence with reversible logic ====
    def apply_persistence(group):
        """
        group: rows for one (y,x), already time-sorted.
        Uses K_RISE to promote to 2 and K_FALL to demote 2 back toward 0.
        """
        states_raw = group["stateraw"].values
        nodata = group["is_nodata"].values


        final = np.zeros(len(group), dtype=np.int8)
        current_state = 0  # 0, 1, or 2
        run_rise = 0
        run_fall = 0


        for i in range(len(group)):
            if nodata[i]:
                # Do not change current_state on nodata; just propagate
                final[i] = current_state
                continue


            if states_raw[i] == 1:
                # Rising anomaly
                run_rise += 1
                run_fall = 0
                if current_state < 2 and run_rise >= K_RISE:
                    current_state = 2
                elif current_state == 0:
                    # transient anomaly, not yet persistent
                    current_state = 1
            else:
                # No anomaly this step
                run_rise = 0
                # If currently in 2, count clean steps to allow demotion
                if current_state == 2:
                    run_fall += 1
                    if run_fall >= K_FALL:
                        current_state = 1  # or 0 if you want full reset
                elif current_state == 1:
                    # If anomaly disappears, slowly decay to 0
                    run_fall += 1
                    if run_fall >= K_FALL:
                        current_state = 0
                else:
                    run_fall = 0


            final[i] = current_state


        return pd.Series(final, index=group.index)


    df["state"] = (
        df.sort_values("time")
        .groupby(["y", "x"], group_keys=False)
        .apply(apply_persistence)
    )


    # Excavated mask is state == 2
    df["excavated"] = (df["state"] == 2).astype(np.uint8)


    # ==== 5. Build xarray state / excavation maps ====
    statemaps = {}
    excavationmaps = {}


    for t, dft in df.groupby("time"):
        # Important: keep original y,x coordinates as in monitordf
        statemaps[t] = (
            dft.set_index(["y", "x"])["state"]
            .to_xarray()
            .rename("state")
        )
        excavationmaps[t] = (
            dft.set_index(["y", "x"])["excavated"]
            .to_xarray()
            .rename("excavated")
        )


    return df, statemaps, excavationmaps

In [7]:
monitor_df = pd.read_csv("results/monitoring_anomaly_scores.csv")

monitor_df, state_maps, excavation_maps = run_excavation_pipeline(
    monitor_df,
    window=3,
    anomaly_quantile=0.90,
    #persistence_days=30
)


/tmp/ipykernel_26805/3860329721.py:146: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(apply_persistence)
